In [1]:
from datasets import load_dataset
import transformers
from transformers import DistilBertTokenizer
from transformers import TrainingArguments
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import tqdm as notebook_tqdm
import torch
import accelerate


/Users/leosteiner/Desktop/BT/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# raw_dataset = load_dataset("imdb")
# print (raw_dataset["test"])

# small_train = raw_dataset["train"].shuffle(seed=42).select(range(1000))
# small_eval = raw_dataset["test"].shuffle(seed=42).select(range(1000))


#### Only load small portion for testing

raw_dataset = load_dataset("imdb", split={"train": "train[:1000]", "test": "test[:1000]"})
print(raw_dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})


In [3]:
# Initzialite tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [11]:
def tokenize_function(example):
    return tokenizer(
        example["text"],       # the review text
        padding="max_length",  # pad to fixed length
        truncation=True,       # truncate if too long
        max_length=512         # BERT/DistilBERT limit
    )


In [12]:
encoded_dataset = raw_dataset.map(tokenize_function, batched=True)
print(encoded_dataset['train'][0])

Map: 100%|██████████| 1000/1000 [00:02<00:00, 440.64 examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [13]:
# Load DistilBERT model for classification
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
############ Check Device ###########
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA GPU")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple Silicon GPU (MPS)")
else:
    device = torch.device("cpu")
    print("Using CPU")
#####################################

Using Apple Silicon GPU (MPS)


In [15]:
training_args = TrainingArguments(
    output_dir="./results",# Directory for saving results
    eval_strategy="epoch",# Evaluate at the end of each epoch
    learning_rate=5e-5,# Initial learning rate
    per_device_train_batch_size=16,# Batch size per GPU
    num_train_epochs=3,# Number of epochs
    weight_decay=0.01,# Regularization
    logging_dir="./logs",# Directory for logs
    logging_steps=10# Log every 10 steps
)

In [16]:
trainer = Trainer(
    model=model,                          # The DistilBERT model
    args=training_args,                   # Training arguments
    train_dataset=encoded_dataset['train'],  # Training data
    eval_dataset=encoded_dataset['test']  # Validation data
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000500,0.000323
2,0.000200,0.000161
3,0.000200,0.000131


/Users/leosteiner/Desktop/BT/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=189, training_loss=0.012888795133216908, metrics={'train_runtime': 133.9441, 'train_samples_per_second': 22.397, 'train_steps_per_second': 1.411, 'total_flos': 397402195968000.0, 'train_loss': 0.012888795133216908, 'epoch': 3.0})

In [17]:
trainer.save_model("./distilbert-imdb-final")
